In [200]:
import pandas as pd
import numpy as np
import math
import xgboost as xgb
import os
import random
import tensorflow as tf
import sklearn.metrics as metrics


from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from arch import arch_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping

In [201]:
# Set a seed value
seed_value= 40 
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

data = pd.read_csv('features.csv')

data.drop_duplicates(subset='Date', keep='last', inplace=True)
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values('Date', inplace=True)
data.set_index('Date', inplace=True)

data = data.asfreq('D') 
data.fillna(method='ffill', inplace=True) 

data['DCOILBRENTEU'] = data['DCOILBRENTEU'].replace('.', np.nan)
data['DCOILBRENTEU'] = data['DCOILBRENTEU'].ffill()
data['DCOILBRENTEU'] = data['DCOILBRENTEU'].astype(float)
data['DCOILBRENTEU_ret'] = data['DCOILBRENTEU'].pct_change()

data['DJIA_ret'] = data['Dow Jones Industrial Average Close'].pct_change()

data['daily_vol_t+1'] = data['daily_vol'].shift(-1)
data = data.drop(data.index[-1])
data = data.drop(data.index[0])

missing_values_count = data.isna().sum()
columns_with_missing_values = missing_values_count[missing_values_count > 0].index.tolist()
columns_with_full_data = missing_values_count[missing_values_count == 0].index.tolist()

print("Columns with missing values:", columns_with_missing_values,'\n')
print("Columns with full values:")

for col in columns_with_full_data:
    print(col)
    

Columns with missing values: ['count', 'sentiment', 'news_count_after_close', 'bearish', 'bullish', 'neutral'] 

Columns with full values:
Unnamed: 0
last_last_hour_ret
last_daily_ret
last_last_hour_whole_day_ret_ratio
last_skewness_last_hour
last_kurtosis_last_hour
last_skewness_day
last_kurtosis_day
last_daily_ret_to_mean_5day_ret
last_skew_ret
last_kurt_ret
last_daily_ret_lag1
last_daily_ret_lag2
last_daily_ret_lag3
last_daily_ret_lag4
last_daily_ret_lag5
high_last_hour_ret
high_daily_ret
high_last_hour_whole_day_ret_ratio
high_skewness_last_hour
high_kurtosis_last_hour
high_skewness_day
high_kurtosis_day
high_daily_ret_to_mean_5day_ret
high_skew_ret
high_kurt_ret
high_daily_ret_lag1
high_daily_ret_lag2
high_daily_ret_lag3
high_daily_ret_lag4
high_daily_ret_lag5
mean_last_hour_ret
mean_daily_ret
mean_last_hour_whole_day_ret_ratio
mean_skewness_last_hour
mean_kurtosis_last_hour
mean_skewness_day
mean_kurtosis_day
mean_daily_ret_to_mean_5day_ret
mean_skew_ret
mean_kurt_ret
mean_daily_

In [202]:
# Replace these with your actual feature column names
target_feature = ['daily_vol_t+1']
vol_features = ['daily_vol','daily_vol_lag1','daily_vol_lag2','daily_vol_lag3','daily_vol_lag4','daily_vol_lag5','last_hour_vol']
ret_features = [ 
    'last_daily_ret',
    'last_daily_ret_lag1',
    'last_last_hour_ret',
    'high_last_hour_ret',
    'high_daily_ret',
    'low_last_hour_ret',
    'low_daily_ret',
    'high_daily_ret_lag1',
    'high_daily_ret_lag2',
    'high_daily_ret_lag3',
    'low_daily_ret_lag1',
    'low_daily_ret_lag2',
    'low_daily_ret_lag3',
    'mean_daily_ret_lag1',
    'range',
    'high_last_hour_whole_day_ret_ratio',
    'high_skewness_last_hour',
    'high_kurtosis_last_hour',
    'high_skewness_day']

trading_features = ['Volume','10Y','1Y','4W','beat_expected']

Industry_features = ['DCOILBRENTEU_ret','eps','DJIA_ret','Dow Jones Industrial Average Close']

Sentiment_features = ['VIX_open','VIX_high','VIX_low','VIX_close','count_polygon','news_count_after_close_polygon',
                      'count', 'sentiment', 'news_count_after_close', 'bearish', 'bullish', 'neutral']

Econ_features = ['CPI','FEDFUNDS','CPI_12_month_pct']

features = vol_features + ret_features + trading_features + Industry_features + Sentiment_features + Econ_features

X = data[features+target_feature]
y = y = X.pop("daily_vol_t+1")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### DecisionTreeRegressor

In [203]:
regression_tree_model = DecisionTreeRegressor(random_state=42)
# Fit the model
regression_tree_model.fit(X_train, y_train)

# Predict on the test set
y_pred = regression_tree_model.predict(X_test)

mse_regression = metrics.mean_squared_error(y_test, y_pred, squared=False)
R_square = metrics.r2_score(y_test, y_pred)
print(f"Mean Squared Error for Regression: {mse_regression}")
print(f"R quare for Regression: {R_square}")

Mean Squared Error for Regression: 0.0015487573853537595
R quare for Regression: 0.58796152717555


In [204]:
importances = regression_tree_model.feature_importances_
for i in range(X.shape[1]):
    if importances[i] > 0.000001:
        print(f"Feature {features[i]}: Built-in Importance: {importances[i]}, ")

### XGBRegressor

In [205]:
xgb_model = xgb.XGBRegressor()
parameters = {
    'max_depth': [9,10,11],
    'n_estimators': [85,80],
    'learning_rate': [0.22,0.2]
}

grid_search = GridSearchCV(xgb_model, parameters, cv=4, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test)


mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)


Fitting 4 folds for each of 12 candidates, totalling 48 fits


Best parameters: {'learning_rate': 0.22, 'max_depth': 11, 'n_estimators': 80}
Mean Squared Error: 1.245216326999272e-06
Root Mean Squared Error: 0.0011158926144568177
Mean Absolute Error: 0.0006464991943711124
R-squared: 0.7860975324541251


In [206]:
def evaluate(y_test, y_test_pred):
    """
    Function to calculate RMSE for one-dimensional arrays
    """
    mse = np.mean((y_test - y_test_pred) ** 2)
    rmse = np.sqrt(mse)
    return rmse

rmse = evaluate(y_test, y_test_pred)

MAX_POSSIBLE_RMSE = y_test.std()
normalized_rmse = (1 - rmse / MAX_POSSIBLE_RMSE)

Grade = normalized_rmse
print('The grade for this assignment is ', math.ceil(Grade * 100))

The grade for this assignment is  54
